In [4]:
!rm install1.sh; wget https://raw.githubusercontent.com/HSE-LAMBDA/RheologyReconstruction/colab_try/install1.sh; chmod +x install1.sh;
!rm install2.sh; wget https://raw.githubusercontent.com/HSE-LAMBDA/RheologyReconstruction/colab_try/install2.sh; chmod +x install2.sh;
!rm get_repo.py; wget https://raw.githubusercontent.com/HSE-LAMBDA/RheologyReconstruction/colab_try/get_repo.py; chmod +x get_repo.py;

--2021-03-30 08:53:16--  https://raw.githubusercontent.com/HSE-LAMBDA/RheologyReconstruction/colab_try/install1.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 92 [text/plain]
Saving to: ‘install1.sh’

install1.sh         100%[===================>]      92  --.-KB/s    in 0s      

2021-03-30 08:53:16 (4.49 MB/s) - ‘install1.sh’ saved [92/92]

--2021-03-30 08:53:16--  https://raw.githubusercontent.com/HSE-LAMBDA/RheologyReconstruction/colab_try/install2.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 157 [text/plain]
Saving 

In [ ]:
! ./install1.sh

In [5]:
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:36
🔁 Restarting kernel...


Тут ядро умирает, надо заново

In [ ]:
! ./install2.sh

In [5]:
!python3 get_repo.py

Cloning into 'RheologyReconstruction'...
remote: Enumerating objects: 181, done.
remote: Counting objects: 100% (181/181), done.
remote: Compressing objects: 100% (135/135), done.
remote: Total 181 (delta 60), reused 118 (delta 36), pack-reused 0
Receiving objects: 100% (181/181), 2.43 MiB | 8.68 MiB/s, done.
Resolving deltas: 100% (60/60), done.
From https://github.com/HSE-LAMBDA/RheologyReconstruction
 * branch            dev        -> FETCH_HEAD
hint: Waiting for your editor to close the file... error: unable to start editor 'editor'
Not committing merge; use 'git commit' to complete the merge.
0
0
0
0
0
0
0
0
0
0
0
0
0
0
['RheologyReconstruction/pipeline/dataset.py', 'RheologyReconstruction/pipeline/logger.py', 'RheologyReconstruction/pipeline/metrics.py', 'RheologyReconstruction/pipeline/trainer.py', 'RheologyReconstruction/pipeline/transforms.py', 'RheologyReconstruction/pipeline/utils.py', 'RheologyReconstruction/pipeline/trainer.py', 'RheologyReconstruction/pipeline/dolfin_adjo

In [9]:
import RheologyReconstruction.pipeline.trainer
import torch
import random
import numpy as np
import os

from RheologyReconstruction.pipeline.dataset import SeismogramDataset
from RheologyReconstruction.pipeline.neural_networks.segnet import SegNet_3Head

In [7]:
seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [8]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
ROOT_DIR = "RheologyReconstruction/pipeline"

Загрузить 2_subdomains_zip.zip

In [10]:
from google.colab import files
uploaded = files.upload()

Saving 2_subdomains_zip.zip to 2_subdomains_zip.zip


In [11]:
!unzip 2_subdomains_zip.zip
!mv 2_subdomains RheologyReconstruction/pipeline/dolfin_adjoint/2_subdomains
dataset_path = os.path.join(ROOT_DIR, 'dolfin_adjoint', '2_subdomains')
train_dataset = SeismogramDataset(dataset_path)


Archive:  2_subdomains_zip.zip
   creating: 2_subdomains/
   creating: 2_subdomains/seismograms/
  inflating: 2_subdomains/seismograms/0.npy  
   creating: 2_subdomains/masks/
  inflating: 2_subdomains/masks/0.npy  


In [12]:
from RheologyReconstruction.pipeline.utils import number_of_parameters

model = SegNet_3Head()
number_of_parameters(model)

Total number of parameters: 4567619
Trainable number of parameters: 4567619


In [13]:
from RheologyReconstruction.pipeline.trainer import BaseTrainer

# NB: such format for coordinates is required for dolfin_adjoint to 
# pass grads through indexing. Why? God knows

detector_coords = [(np.array([c, 2000.])) for c in np.linspace(0., 2000., 128)]

# TODO: test with nonempty logger
t = BaseTrainer(model, 
                device, 
                train_dataset,
                optimizer_type=torch.optim.Adam,
                optimizer_params={'lr': 1e-3},
                snapshot_interval=250)

In [14]:
!mkdir dolfin_adjoint
!cp RheologyReconstruction/pipeline/dolfin_adjoint/solver_config.yaml dolfin_adjoint/solver_config.yaml 

In [15]:
t.train(detector_coords, batch_size=1, epochs=100, num_solver_type='adjoint_equation')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Running training procedure:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch 1 of 100:   0%|          | 0/1 [00:00<?, ?it/s]

Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.


KeyboardInterrupt: ignored